In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
#dataset.tail()

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset['age'].hist(bins=11, figsize=(5, 5))

In [ ]:
dataset['age_norm'] = (dataset['age'] - dataset['age'].min()) / (dataset['age'].max() - dataset['age'].min())
dataset

In [ ]:
dataset['sex'].unique()

In [ ]:
#
dataset['sex_enc'] = (dataset['sex'] == 'male').astype(int)
dataset

In [ ]:
dataset['bmi'].hist(bins=12, figsize=(5, 5))

In [ ]:
# Assume a normal distribution
# 生物統計のエッセンス ―その 1― 浦島充佳, 2018
dataset['bmi_norm'] = (dataset['bmi'] - dataset['bmi'].mean()) / dataset['bmi'].std()
dataset.loc[dataset['bmi_norm'] > 3, 'bmi_norm'] = 3
dataset['bmi_norm'] = dataset['bmi_norm'] / 3
dataset

In [ ]:
dataset['bmi_norm'].hist(bins=12, figsize=(5, 5))

In [ ]:
dataset['children_norm'] = (dataset['children'] - dataset['children'].min()) / (dataset['children'].max() - dataset['children'].min())
dataset

In [ ]:
dataset['smoker'].unique()

In [ ]:
dataset['smoker_enc'] = (dataset['smoker'] == 'no').astype(int)
dataset

In [ ]:
dataset['region'].unique()

In [ ]:
dataset = dataset.join(pd.get_dummies(dataset['region']).astype(int))
dataset

In [ ]:
dataset['expenses'].hist(bins=12, figsize=(5, 5))

In [ ]:
dataset.shape

In [ ]:
# create train and test dataset
use_column = ['age_norm', 'bmi_norm', 'smoker_enc', 'sex_enc', 'children_norm']
train_dataset = dataset.loc[:int(dataset.shape[0] * 0.8), use_column]
train_labels = dataset.loc[:int(dataset.shape[0] * 0.8), 'expenses']
test_dataset = dataset.loc[int(dataset.shape[0] * 0.8):, use_column]
test_labels = dataset.loc[int(dataset.shape[0] * 0.8):, 'expenses']

In [ ]:
train_dataset

In [ ]:
train_labels

In [ ]:
test_dataset

In [ ]:
test_labels

In [ ]:
grr = pd.plotting.scatter_matrix(train_dataset.join(train_labels), figsize=(15, 15))

In [ ]:
model = keras.models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(train_dataset.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error'])
history = model.fit(train_dataset.values, train_labels.values, epochs=200)

In [ ]:
print(history.history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
